In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [2]:
#Parameters
file = "../tables/2021AP.xlsx"
next_week = 6
#Read in df
df = pd.read_excel(file)
df

,Team,Week,Rank,W,L,Winning Percentage,Opp. Rank,Opp. P5,Home,Result,Points Scored,Points Against,Margin,Next Week Rank,Movement
0,Alabama,1,1,0,0,1,14,1,0,W,44,13,31,1,0
1,Oklahoma,1,2,0,0,1,26,0,1,W,40,35,5,4,-2
2,Clemson,1,3,0,0,1,5,1,0,L,3,10,-7,6,-3
3,Ohio State,1,4,0,0,1,26,1,0,W,45,31,14,3,1
4,Georgia,1,5,0,0,1,3,1,0,W,10,3,7,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Arizona State,6,22,4,1,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,NC State,6,23,4,1,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,SMU,6,24,5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,San Diego State,6,25,4,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Drop footer
df.drop(index = (df.index.stop - 1), inplace = True)

#Drop null columns (teams with byes)
df.dropna(inplace = True)

#Set week
for index, row in df.iterrows():
    if row['Week'] >= next_week:
        df.drop(index = index, inplace = True)

In [4]:
for index, row in df.iterrows():
    if row['Result'] == 'W':
        df.at[index, 'Result'] = 1
    else: 
        df.at[index, 'Result'] = 0

In [5]:
df = df.drop(columns = ["Movement", "Team"])

In [6]:
df

,Week,Rank,W,L,Winning Percentage,Opp. Rank,Opp. P5,Home,Result,Points Scored,Points Against,Margin,Next Week Rank
0,1,1,0,0,1,14,1,0,1,44,13,31,1
1,1,2,0,0,1,26,0,1,1,40,35,5,4
2,1,3,0,0,1,5,1,0,0,3,10,-7,6
3,1,4,0,0,1,26,1,0,1,45,31,14,3
4,1,5,0,0,1,3,1,0,1,10,3,7,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,5,21,4,0,1,19,1,0,0,14,17,-3,26
121,5,22,3,1,0.75,26,1,0,1,24,19,5,18
122,5,23,3,1,0.75,26,0,1,1,34,27,7,23
123,5,24,4,0,1,26,1,1,1,37,34,3,19


In [7]:
for name, values in df.iteritems():
    df[name] = pd.to_numeric(values)
    
df.dtypes

Week                    int64
Rank                    int64
W                       int64
L                       int64
Winning Percentage    float64
Opp. Rank               int64
Opp. P5                 int64
Home                    int64
Result                  int64
Points Scored           int64
Points Against          int64
Margin                  int64
Next Week Rank          int64
dtype: object

In [8]:
X = df.drop(columns = ['Next Week Rank'])
y = df["Next Week Rank"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 8)
X_train.shape

(90, 12)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
#RF
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(random_state = 8)
rf_model.fit(X_train, y_train)
rf_model.score(X_test, y_test)

0.8827881578114116

In [12]:
#Scaled RF
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(random_state = 8)
rf_model.fit(X_train_scaled, y_train)
rf_model.score(X_test_scaled, y_test)

0.8831353888290799

In [13]:
#Multiple Linear Regression
from sklearn.linear_model import LinearRegression
lin_model = LinearRegression().fit(X_train, y_train)
lin_model.score(X_test, y_test)

0.9412201573377569

In [14]:
results = pd.DataFrame(data = [lin_model.predict(X_test), y_test])
results

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,5.808704,6.114197,13.021706,11.299505,12.884234,12.478461,17.141147,24.102672,8.434996,5.564156,...,10.280536,14.192452,11.005079,3.006307,19.602687,5.633799,29.52623,8.635457,24.05122,1.597502
1,6.000000,7.000000,10.000000,10.000000,16.000000,13.000000,15.000000,26.000000,6.000000,4.000000,...,10.000000,12.000000,10.000000,1.000000,18.000000,3.000000,26.00000,6.000000,26.00000,1.000000


In [15]:
#Scaled Multiple Linear Regression
from sklearn.linear_model import LinearRegression
scaled_lin_model = LinearRegression().fit(X_train_scaled, y_train)
scaled_lin_model.score(X_test_scaled, y_test)

0.941220157337757

In [16]:
results = pd.DataFrame(data = [lin_model.predict(X_test), y_test])
results

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,5.808704,6.114197,13.021706,11.299505,12.884234,12.478461,17.141147,24.102672,8.434996,5.564156,...,10.280536,14.192452,11.005079,3.006307,19.602687,5.633799,29.52623,8.635457,24.05122,1.597502
1,6.000000,7.000000,10.000000,10.000000,16.000000,13.000000,15.000000,26.000000,6.000000,4.000000,...,10.000000,12.000000,10.000000,1.000000,18.000000,3.000000,26.00000,6.000000,26.00000,1.000000


In [17]:
#AdaBoost Regression
from sklearn.ensemble import AdaBoostRegressor as abr
ab_model = abr(random_state = 8).fit(X_train, y_train)
ab_model.score(X_test, y_test)

0.8984665393320708

In [18]:
#Scaled AdaBoost Regression
from sklearn.ensemble import AdaBoostRegressor as abr
ab_model = abr(random_state = 8).fit(X_train_scaled, y_train)
ab_model.score(X_test_scaled, y_test)

0.9137352592737905